In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes datasets bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.2/806.2 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 79.2 MB/s eta 0:00:00
  Installing build d

In [19]:
from datasets import load_dataset
import pandas as pd

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired

# # We select a subsample of 5000 abstracts from ArXiv
# # dataset = load_dataset("CShorten/ML-ArXiv-Papers")["train"]
# # docs = dataset["abstract"][:1_00]
# dataset = load_dataset("sp01/Automotive_NER")['train']
# # Drop rows with missing values
# dataset.dropna(subset=['NOTES'], inplace=True)
# docs = dataset['NOTES'][:1_000]


# Load the dataset
dataset = load_dataset("sp01/Automotive_NER")['train']
# Convert the dataset into a pandas DataFrame
df = pd.DataFrame(dataset)
# Drop rows with missing values in the 'NOTES' column
df.dropna(subset=['DESC_DEFECT'], inplace=True)
# Select a subset of the dataset
docs = df['DESC_DEFECT'][:1000]

# We define a number of topics that we know are in the documents
zeroshot_topic_list = ["component", "failure issue", "vehicle model", "corrective action"]

# We fit our model using the zero-shot topics
# and we define a minimum similarity. For each document,
# if the similarity does not exceed that value, it will be used
# for clustering instead.
topic_model = BERTopic(
    embedding_model="thenlper/gte-small",
    min_topic_size=15,
    zeroshot_topic_list=zeroshot_topic_list,
    zeroshot_min_similarity=.85,
    representation_model=KeyBERTInspired()
)
topics, _ = topic_model.fit_transform(docs)


In [20]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,6,vehicle model,"[vehicles, vehicle, minivans, braking, , , , ,...",[VEHICLE DESCRIPTION: SPORT UTILITY VEHICLES ...
1,1,2,failure issue,"[wiper, windshield, motor, vehicles, , , , , , ]",[CERTAIN SPORT UTILITY AND PASSENGER VEHICLES ...
2,2,87,-1_transmission_shift_malfunctions_trans,"[transmission, shift, malfunctions, trans, sel...","[ON CERTAIN MOTOR HOMES AND TOUR COACHES, A 'C..."
3,3,59,0_trucks_vehicles_safety_vehicle,"[trucks, vehicles, safety, vehicle, fail, lamp...",[CERTAIN HEAVY DUTY CLASS 8 TRUCKS FAIL TO COM...
4,4,55,1_axles_recall_trucks_bearings,"[axles, recall, trucks, bearings, fasteners, a...",[HEAVY DUTY TRUCKS EQUIPPED WITH TRW TIE ROD E...
5,5,54,2_vehicles_truck_braking_safety,"[vehicles, truck, braking, safety, brake, vehi...","[CERTAIN FLATBED, DUMP, TRUCK DOLLY, UNLOADER,..."
6,6,52,3_tires_incorrectly_safety_miscalculations,"[tires, incorrectly, safety, miscalculations, ...",[CERTAIN MOTOR HOMES FAIL TO COMPLY WITH THE R...
7,7,46,4_valves_valve_brake_exhaust,"[valves, valve, brake, exhaust, trailers, sr02...",[ON CERTAIN TRAILERS EQUIPPED WITH BENDIX SR-2...
8,8,43,5_buses_bus_transit_exits,"[buses, bus, transit, exits, , , , , , ]",[CERTAIN TRANSIT BUSES FAIL TO COMPLY WITH THE...
9,9,39,6_djg___,"[djg, , , , , , , , , ]",[ON CERTAIN DOREL JUVENILE GROUP (DJG) CHILD R...
